In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
from sqlalchemy import create_engine
from config import postgres_conn_str
import requests

In [2]:
#url to be scraped
url = 'https://www.heritage.org/index/ranking'

### Splinter Method

In [3]:
# #use chrome as browser and visit url/parse html from url
# executable_path = {'executable_path': 'chromedriver.exe'}
# browser = Browser('chrome', **executable_path, headless=True)
# browser.visit(url)
# html = browser.html
# soup = BeautifulSoup(html, 'html.parser')

### Request Method

In [3]:
# Retrieve page with the requests module and parse html from url
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [6]:
#find all table data
results = soup.find_all('td')

#create empty lists
country_list, overall_list, change_list = ([] for i in range(3))

#append result to a list depending on the td class is associated with
for result in results:
    text = result.text.strip()
    if 'country' in result['class']:
        country_list.append(text)
    elif 'overall' in result['class']:
        overall_list.append(text)
    elif 'change' in result['class']:
        change_list.append(text)

#convert to dataframe
dict = {'country': country_list,'overall': overall_list,'change': change_list}
rankings_df = pd.DataFrame(dict)

In [7]:
rankings_df.head()

,country,overall,change
0,Singapore,89.4,0.0
1,Australia,82.6,+1.7
2,Hong Kong,89.1,-1.1
3,Switzerland,82.0,+0.1
4,New Zealand,84.1,-0.3


In [7]:
#create database connection
engine = create_engine(f'postgresql://{postgres_conn_str}')

In [8]:
#insert df into postgres
rankings_df.to_sql(name='rankings', con=engine, if_exists='replace', index=False)